### Q1. What is Gradient Boosting Regression?
Ans. Gradient Boosting Regression is a popular machine learning technique used for regression tasks. It is an extension of the gradient boosting algorithm, which is an ensemble learning method that combines multiple weak learners (usually decision trees) to create a stronger predictive model. In Gradient Boosting Regression, weak learners are trained sequentially, and each subsequent learner aims to correct the errors made by the previous ones.

The basic idea behind Gradient Boosting Regression is to fit the weak learners to the negative gradient (or the "residuals") of the loss function with respect to the predictions of the previous model. This way, each new model focuses on learning the remaining patterns in the data that the previous models were unable to capture.

### Q2. Implement a simple gradient boosting algorithm from scratch using Python and NumPy. Use a simple regression problem as an example and train the model on a small dataset. Evaluate the model's performance using metrics such as mean squared error and R-squared.

In [5]:
import numpy as np

class GradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        # Initialize the first model with the mean of the target values
        self.models.append(np.mean(y))
        residuals = y - self.models[0]
        
        for i in range(self.n_estimators):
            # Calculate negative gradients (residuals) for the current model
            gradients = -residuals

            # Train a decision tree on the negative gradients
            tree = self.build_tree(X, gradients, depth=0)
            self.models.append(tree)

            # Update the residuals with the predictions from the new tree
            residuals -= self.learning_rate * self.predict_tree(X, tree)

    def predict(self, X):
        # Make predictions using the ensemble of weak learners
        predictions = np.zeros(len(X))
        for model in self.models[1:]:
            predictions += self.learning_rate * self.predict_tree(X, model)
        return predictions + self.models[0]

    def predict_tree(self, X, tree):
        if isinstance(tree, tuple):
            feature_index, threshold, left_tree, right_tree = tree
            predictions = np.where(X[:, feature_index] < threshold, 
                                   self.predict_tree(X, left_tree),
                                   self.predict_tree(X, right_tree))
            return predictions
        else:
            return np.full(len(X), tree)

    def build_tree(self, X, y, depth):
        if depth == self.max_depth or len(X) == 1:
            return np.mean(y)

        feature_index, threshold, split_loss = self.find_best_split(X, y)

        left_mask = X[:, feature_index] < threshold
        right_mask = ~left_mask

        left_tree = self.build_tree(X[left_mask], y[left_mask], depth+1)
        right_tree = self.build_tree(X[right_mask], y[right_mask], depth+1)

        return (feature_index, threshold, left_tree, right_tree)

    def find_best_split(self, X, y):
        best_feature_index, best_threshold, best_split_loss = None, None, float('inf')

        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] < threshold
                right_mask = ~left_mask

                left_loss = np.sum((y[left_mask] - np.mean(y[left_mask]))**2)
                right_loss = np.sum((y[right_mask] - np.mean(y[right_mask]))**2)
                split_loss = left_loss + right_loss

                if split_loss < best_split_loss:
                    best_feature_index = feature_index
                    best_threshold = threshold
                    best_split_loss = split_loss

        return best_feature_index, best_threshold, best_split_loss

def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

def r_squared(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (ss_res / ss_tot)

# Sample data
X_train = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)
y_train = np.array([2, 4, 5, 4, 5])

# Train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
gb_regressor.fit(X_train, y_train)

# Test data
X_test = np.array([6, 7, 8]).reshape(-1, 1)

# Make predictions
y_pred = gb_regressor.predict(X_test)

# Evaluate the model's performance
print("Mean Squared Error:", mean_squared_error(y_train, gb_regressor.predict(X_train)))
print("R-squared:", r_squared(y_train, gb_regressor.predict(X_train)))

Mean Squared Error: 208895804.20650816
R-squared: -174079835.8387568


### Q3. Experiment with different hyperparameters such as learning rate, number of trees, and tree depth to optimise the performance of the model. Use grid search or random search to find the best hyperparameters.

In [8]:
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

class GradientBoostingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.models = []

    def fit(self, X, y):
        # Initialize the first model with the mean of the target values
        self.models.append(np.mean(y))
        residuals = y - self.models[0]
        
        for i in range(self.n_estimators):
            # Calculate negative gradients (residuals) for the current model
            gradients = -residuals

            # Train a decision tree on the negative gradients
            tree = self.build_tree(X, gradients, depth=0)
            self.models.append(tree)

            # Update the residuals with the predictions from the new tree
            residuals -= self.learning_rate * self.predict_tree(X, tree)

    def predict(self, X):
        # Make predictions using the ensemble of weak learners
        predictions = np.zeros(len(X))
        for model in self.models[1:]:
            predictions += self.learning_rate * self.predict_tree(X, model)
        return predictions + self.models[0]

    def predict_tree(self, X, tree):
        if isinstance(tree, tuple):
            feature_index, threshold, left_tree, right_tree = tree
            predictions = np.where(X[:, feature_index] < threshold, 
                                   self.predict_tree(X, left_tree),
                                   self.predict_tree(X, right_tree))
            return predictions
        else:
            return np.full(len(X), tree)

    def build_tree(self, X, y, depth):
        if depth == self.max_depth or len(X) == 1:
            return np.mean(y)

        feature_index, threshold, split_loss = self.find_best_split(X, y)

        left_mask = X[:, feature_index] < threshold
        right_mask = ~left_mask

        left_tree = self.build_tree(X[left_mask], y[left_mask], depth+1)
        right_tree = self.build_tree(X[right_mask], y[right_mask], depth+1)

        return (feature_index, threshold, left_tree, right_tree)

    def find_best_split(self, X, y):
        best_feature_index, best_threshold, best_split_loss = None, None, float('inf')

        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                left_mask = X[:, feature_index] < threshold
                right_mask = ~left_mask

                left_loss = np.sum((y[left_mask] - np.mean(y[left_mask]))**2)
                right_loss = np.sum((y[right_mask] - np.mean(y[right_mask]))**2)
                split_loss = left_loss + right_loss

                if split_loss < best_split_loss:
                    best_feature_index = feature_index
                    best_threshold = threshold
                    best_split_loss = split_loss

        return best_feature_index, best_threshold, best_split_loss

# Sample data
X_train = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)
y_train = np.array([2, 4, 5, 4, 5])

# Define the Gradient Boosting Regressor with default hyperparameters
gb_regressor = GradientBoostingRegressor()

# Define hyperparameters to search through
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4]
}

# Perform grid search with 3-fold cross-validation
grid_search = GridSearchCV(gb_regressor, param_grid, scoring='neg_mean_squared_error', cv=3)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

# Test data
X_test = np.array([6, 7, 8]).reshape(-1, 1)
y_test = np.array([6, 7, 8])

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model's performance on the test data
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (Test):", mse)
print("R-squared (Test):", r2)

Best Hyperparameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50}
Mean Squared Error (Test): 11.704449678631754
R-squared (Test): -16.55667451794763


### Q4. What is a weak learner in Gradient Boosting?
Ans. In Gradient Boosting, a weak learner refers to a simple and relatively low-complexity model that performs slightly better than random guessing on the given training data. Typically, a weak learner is a decision tree with a small depth (often referred to as a "decision stump" if the depth is limited to only one split).

The idea behind using weak learners in Gradient Boosting is that combining multiple weak learners can lead to a powerful ensemble model. Each weak learner focuses on learning the patterns that are not captured by the previous models, and their predictions are combined to form the final prediction of the ensemble.

Using weak learners allows Gradient Boosting to iteratively learn and correct errors made by the ensemble so far, making the final model more accurate and robust.

### Q5. What is the intuition behind the Gradient Boosting algorithm?
Ans. The intuition behind the Gradient Boosting algorithm is to create a strong predictive model by combining multiple weak learners (usually decision trees) sequentially. The idea is to iteratively correct the errors made by the previous weak learners, ultimately leading to a powerful ensemble model. The key intuition can be summarized as follows:

    Starting Point: The algorithm starts with a simple model that serves as the initial prediction for all instances in the training data. For regression tasks, this can be the mean of the target variable, while for classification tasks, it can be the majority class.
    Error Correction: The algorithm then sequentially adds weak learners (e.g., decision trees) to the ensemble. Each weak learner is trained to focus on learning the patterns in the data that the previous models failed to capture. It does this by fitting to the negative gradients (residuals) of the loss function with respect to the current ensemble's predictions.
    Weighted Combination: The predictions of all weak learners are combined in a weighted manner, where the weight assigned to each learner depends on its individual performance. More accurate learners are given higher weights in the ensemble.
    Iterative Learning: The process continues for a predefined number of iterations or until a stopping criterion is met. Each subsequent weak learner emphasizes the mistakes made by the previous ensemble, leading to a more accurate and robust model.

By combining multiple weak learners in an iterative manner, the Gradient Boosting algorithm can effectively learn complex relationships in the data and provide excellent predictive performance.

### Q6. How does Gradient Boosting algorithm build an ensemble of weak learners?
Ans. The Gradient Boosting algorithm builds an ensemble of weak learners through an iterative process. The steps involved in constructing the ensemble can be summarized as follows:

    Start with a simple model: The process begins by initializing the ensemble with a simple model, which is usually the mean of the target variable (for regression) or the majority class (for classification). This initial model serves as the baseline prediction.
    Calculate negative gradients: For each iteration, the algorithm calculates the negative gradients (also known as the "residuals") of the loss function with respect to the current ensemble's predictions. These gradients represent the errors made by the current ensemble on the training data.
    Train a weak learner: A weak learner (e.g., decision tree) is trained on the negative gradients obtained in the previous step. The weak learner's goal is to learn and capture the patterns in the data that the current ensemble failed to model accurately.
    Update the ensemble: The weak learner is added to the ensemble with a weight proportional to its performance (usually based on its accuracy). The weight determines the influence of the weak learner in the final prediction. Additionally, the predictions of the weak learner are combined with the predictions of the previous ensemble using the learning rate, which controls the step size of the iterative learning process.
    Iterative learning: Steps 2 to 4 are repeated for a predefined number of iterations or until a stopping criterion is met. With each iteration, the weak learners focus on correcting the errors made by the current ensemble, leading to an ensemble that continually improves its predictive performance.

By iteratively combining the predictions of weak learners and emphasizing the mistakes made by the ensemble so far, the Gradient Boosting algorithm constructs a powerful ensemble that can generalize well on the data and achieve high predictive accuracy.

### Q7. What are the steps involved in constructing the mathematical intuition of Gradient Boosting algorithm?
Ans. The mathematical intuition of the Gradient Boosting algorithm can be understood through the following steps:

    Loss function: Define a differentiable loss function that measures the difference between the predicted values and the true target values. For example, for regression tasks, the commonly used loss function is the Mean Squared Error (MSE), and for binary classification tasks, it is the Log Loss (or Cross-Entropy Loss).
    Initialize the ensemble: Initialize the ensemble with a simple model, which serves as the initial prediction for all instances in the training data. For regression tasks, this can be the mean of the target variable, while for binary classification tasks, it can be the log-odds (logit) of the positive class.
    Negative gradients (residuals): Calculate the negative gradients (residuals) of the loss function with respect to the current ensemble's predictions. These gradients represent the errors made by the current ensemble on the training data.
    Train a weak learner: Train a weak learner (e.g., decision tree) on the negative gradients obtained in the previous step. The weak learner's goal is to learn and capture the patterns in the data that the current ensemble failed to model accurately.
    Weighted addition to the ensemble: Add the trained weak learner to the ensemble with a weight proportional to its performance (e.g., based on its accuracy). The weight determines the influence of the weak learner in the final prediction.
    Update predictions: Update the predictions of the ensemble by adding the weighted predictions of the new weak learner. The learning rate controls the step size of the iterative learning process and balances the influence of each new weak learner.
    Iterative learning: Repeat steps 3 to 6 for a predefined number of iterations or until a stopping criterion is met. During each iteration, the weak learners focus on correcting the errors made by the current ensemble, leading to an ensemble that continually improves its predictive performance.
    Final prediction: The final prediction of the Gradient Boosting ensemble is the weighted sum of the predictions made by all the weak learners in the ensemble.

By iteratively combining the predictions of weak learners and updating the ensemble based on the negative gradients of the loss function, the Gradient Boosting algorithm constructs a powerful ensemble model that can generalize well and achieve high predictive accuracy.
